<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=114c0faeea5a29819e89f79399d1bdb09b1f0c2c47ebbb789136a498cf8c8cae
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.1
    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-25 14:28:14
-------------------
qualified stocks: 87
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  400.00
Current:  1.42 C
-------------------
Today PnL: -15.97 K (-0.11%)
Current PnL: -23.53 L (-15.42%)
CY Booked + Current PnL: -9.64 L (-6.32%)
-------------------
Total profit:  1.30 L
Total loss:  -24.83 L
-------------------
Total Booked + Current PnL: 17.34 L (13.8%)
Total Booked PnL: 40.87 L (32.53%)
Curr Year Booked PnL: 13.89 L (9.76%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.31 C
Est FTT PnL: 88.94 L (62.52%)
Deployed:  1.26 C
Current:  1.42 C
CAGR/XIRR %: 7.07%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.00,0.20,-14.05,16.45,0.09,14248.0,-14160.0,86617.0,101.07,50.0,M-SC,3.98,253.0,-0.99,0.61,13.80,OX40N,NTT,DURABLES
39,INDIGOPNTS,1408.00,-0.08,-9.40,10.40,0.02,16447.0,-16416.0,158143.0,141.07,70.0,M-SC,6.66,234.0,-1.00,1.11,37.56,OX40N,NTT,PAINTS
35,ICICIGI,2252.93,-0.33,6.57,12.79,20.19,23760.0,11448.0,185768.0,-16.49,50.0,X-MC,7.63,68.0,0.48,1.31,22.67,X40,ATH,INSURANCE
43,ITC,452.00,-0.21,-1.61,12.24,10.44,24103.0,-3218.0,196920.0,-41.15,42.0,X-LC,0.83,5.0,-0.13,1.38,3.87,X40,NTT,FMCG
53,NESTLEIND,1377.00,-0.16,11.72,8.66,21.40,25679.0,31099.0,296525.0,3.66,54.0,X-LC,7.79,12.0,1.21,2.08,19.14,XY25,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,424.00,4.50,-25.94,92.87,42.84,44712.0,-16861.0,48145.0,-51.52,45.0,X-SC,37.29,83.0,-0.38,0.34,4.76,XY24,NTT,MISC
62,ROUTE,2227.21,3.68,-49.48,234.62,69.05,157726.0,-65840.0,67226.0,-58.98,37.0,H-SC,23.56,140.0,-0.42,0.47,3.68,SR,ATH,IT
61,REPCOHOME,880.00,2.64,-10.70,112.56,89.82,287987.0,-30653.0,255852.0,-55.29,50.0,H-SC,10.43,160.0,-0.11,1.80,35.36,XY24,NTT,FINANCE
58,RAJOOENG,143.10,2.31,-29.18,97.13,39.61,70507.0,-29910.0,72590.0,-52.19,25.0,H-SC,20.03,135.0,-0.42,0.51,2.31,AR,ATH,MISC
65,SHALBY,327.00,2.24,-18.51,53.51,25.10,78989.0,-33527.0,147616.0,986.12,33.0,M-SC,16.29,250.0,-0.42,1.04,18.30,XY24,NTT,HEALTHCARE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
34,HONAUT,58357.33,-3.49,-14.04,66.74,43.32,93436.0,-22872.0,140000.0,-26.09,43.0,X-SC,2.01,90.0,-0.24,0.98,7.49,X40N,ATH,ELECTRICAL
3,ACC,3906.00,-2.78,-21.81,109.94,64.15,204543.0,-51901.0,186050.0,-54.13,52.0,X-SC,1.66,82.0,-0.25,1.31,4.36,XY24,BTT,CEMENT
57,RAJESHEXPO,518.00,-2.18,-63.83,176.64,0.07,87651.0,-87556.0,49621.0,1718.18,53.0,L-SC,1.53,268.0,-1.00,0.35,23.41,OX40N,NTT,JEWELLERY
8,AWL,485.00,-2.02,-13.92,78.31,53.50,203418.0,-41991.0,259760.0,-58.27,50.0,X-MC,4.23,58.0,-0.21,1.83,15.38,XY24,NTT,FMCG
29,HAPPSTMNDS,1480.71,-1.78,-37.84,197.39,84.85,157249.0,-48501.0,79664.0,-26.17,42.0,H-SC,14.22,147.0,-0.31,0.56,2.24,AR,ATH,IT


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,1.31,0.46,105.78,106.72,131059.0,562.0,123898.0,-50.25,38.0,H-SC,3.71,139.0,0.00,0.87,23.94,AR,ATH,MISC
85,WIPRO,420.00,-0.67,0.74,71.00,72.27,121196.0,1251.0,170699.0,-13.34,53.0,M-LC,4.55,99.0,0.01,1.20,6.96,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,0.26,-5.03,118.93,107.91,166755.0,-7431.0,140213.0,-22.58,34.0,M-SC,10.47,216.0,-0.04,0.99,0.26,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-0.08,-9.40,10.40,0.02,16447.0,-16416.0,158143.0,141.07,70.0,M-SC,6.66,234.0,-1.00,1.11,37.56,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,0.20,-14.05,16.45,0.09,14248.0,-14160.0,86617.0,101.07,50.0,M-SC,3.98,253.0,-0.99,0.61,13.80,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-0.56,-23.01,47.38,13.47,98375.0,-62037.0,207630.0,-68.31,25.0,H-SC,1.38,158.0,-0.63,1.46,5.57,XY24,NTT,PAINTS
44,JCHAC,2282.00,-0.36,-23.48,30.67,-0.01,29995.0,-30007.0,97798.0,19921.74,55.0,M-SC,1.67,233.0,-1.00,0.69,10.74,OX40N,NTT,AC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.00,-0.67,0.74,71.00,72.27,121196.0,1251.0,170699.0,-13.34,53.0,M-LC,4.55,99.0,0.01,1.20,6.96,XR,NTT,IT
38,INDIAMART,4810.62,1.31,0.46,105.78,106.72,131059.0,562.0,123898.0,-50.25,38.0,H-SC,3.71,139.0,0.00,0.87,23.94,AR,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.00,-0.67,0.74,71.00,72.27,121196.0,1251.0,170699.0,-13.34,53.0,M-LC,4.55,99.0,0.01,1.20,6.96,XR,NTT,IT
38,INDIAMART,4810.62,1.31,0.46,105.78,106.72,131059.0,562.0,123898.0,-50.25,38.0,H-SC,3.71,139.0,0.00,0.87,23.94,AR,ATH,MISC
25,FINCABLES,1641.55,0.26,-5.03,118.93,107.91,166755.0,-7431.0,140213.0,-22.58,34.0,M-SC,10.47,216.0,-0.04,0.99,0.26,OX40N,ATH,CABLES
86,ZYDUSLIFE,1286.17,0.43,-1.87,38.21,35.63,78584.0,-3911.0,205663.0,-15.85,36.0,H-MC,4.38,119.0,-0.05,1.45,14.49,AR,ATH,PHARMA
84,WHIRLPOOL,2270.00,0.74,-4.49,90.47,81.92,99197.0,-5152.0,109646.0,-44.03,35.0,M-SC,4.56,236.0,-0.05,0.77,29.80,XR,NTT,DURABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TMPV,600.00,-1.33,-16.22,69.71,42.18,161431.0,-44835.0,231575.0,-24.30,20.0,X-LC,0.63,3.0,-0.28,1.63,0.00,XY24,NTT,AUTO
46,JSWINFRA,342.00,-0.69,-9.99,28.02,15.23,50527.0,-20021.0,180326.0,-22.98,23.0,X-MC,6.67,66.0,-0.40,1.27,19.03,X40N,NTT,REALTY
11,BATAINDIA,2096.00,-0.38,-35.37,109.20,35.20,90809.0,-45512.0,83158.0,9.03,24.0,X-SC,12.88,92.0,-0.50,0.58,0.15,X40,NTT,FOOTWEAR
60,RELAXO,1176.00,-0.56,-48.11,193.52,52.31,145763.0,-69838.0,75322.0,-44.73,26.0,X-SC,3.97,91.0,-0.48,0.53,0.82,X40N,NTT,FOOTWEAR
22,DIXON,18931.72,-1.15,-6.88,30.56,21.58,39881.0,-9639.0,130500.0,-53.04,29.0,X-MC,6.87,56.0,-0.24,0.92,15.53,X40N,ATH,IT


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TMPV,600.0,-1.33,-16.22,69.71,42.18,161431.0,-44835.0,231575.0,-24.30,20.0,X-LC,0.63,3.0,-0.28,1.63,0.00,XY24,NTT,AUTO
43,ITC,452.0,-0.21,-1.61,12.24,10.44,24103.0,-3218.0,196920.0,-41.15,42.0,X-LC,0.83,5.0,-0.13,1.38,3.87,X40,NTT,FMCG
36,ICICIPRULI,790.0,0.74,1.43,29.12,30.97,52204.0,2534.0,179272.0,-22.41,50.0,X-MC,1.57,75.0,0.05,1.26,14.17,X40,ATH,INSURANCE
3,ACC,3906.0,-2.78,-21.81,109.94,64.15,204543.0,-51901.0,186050.0,-54.13,52.0,X-SC,1.66,82.0,-0.25,1.31,4.36,XY24,BTT,CEMENT
66,SIEMENS,4671.5,0.19,-14.83,47.37,25.51,75079.0,-27600.0,158495.0,-2.66,53.0,H-LC,1.87,49.0,-0.37,1.11,16.80,AR,ATH,ELECTRICAL


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,-0.62,-14.04,72.69,48.45,125291.0,-28142.0,172364.0,-20.30,29.0,X-MC,6.83,64.0,-0.22,1.21,0.00,X40N,ATH,FINANCE
54,PAGEIND,51605.07,-0.93,-5.95,34.04,26.07,52422.0,-9740.0,154000.0,-30.68,33.0,X-MC,9.06,55.0,-0.19,1.08,0.00,X40,ATH,APPARELS
76,TMPV,600.00,-1.33,-16.22,69.71,42.18,161431.0,-44835.0,231575.0,-24.30,20.0,X-LC,0.63,3.0,-0.28,1.63,0.00,XY24,NTT,AUTO
11,BATAINDIA,2096.00,-0.38,-35.37,109.20,35.20,90809.0,-45512.0,83158.0,9.03,24.0,X-SC,12.88,92.0,-0.50,0.58,0.15,X40,NTT,FOOTWEAR
55,PGHH,17907.65,-0.33,-4.44,39.98,33.76,76720.0,-8925.0,191895.0,-32.64,33.0,X-MC,3.36,57.0,-0.12,1.35,0.52,X40,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,424.00,4.50,-25.94,92.87,42.84,44712.0,-16861.0,48145.0,-51.52,45.0,X-SC,37.29,83.0,-0.38,0.34,4.76,XY24,NTT,MISC
60,RELAXO,1176.00,-0.56,-48.11,193.52,52.31,145763.0,-69838.0,75322.0,-44.73,26.0,X-SC,3.97,91.0,-0.48,0.53,0.82,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.38,-35.37,109.20,35.20,90809.0,-45512.0,83158.0,9.03,24.0,X-SC,12.88,92.0,-0.50,0.58,0.15,X40,NTT,FOOTWEAR
22,DIXON,18931.72,-1.15,-6.88,30.56,21.58,39881.0,-9639.0,130500.0,-53.04,29.0,X-MC,6.87,56.0,-0.24,0.92,15.53,X40N,ATH,IT
52,MEDANTA,1486.00,-0.66,3.72,19.63,24.08,25847.0,4723.0,131673.0,-4.69,47.0,X-SC,4.93,89.0,0.18,0.93,24.07,XY24,NTT,HEALTHCARE


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4389.97,-0.73,-15.27,40.78,19.28,119538.0,-52826.0,293130.0,-26.21,57.0,X-LC,7.53,1.0,-0.44,2.06,8.37,X40,ATH,IT
41,INFY,2275.00,0.19,5.38,46.96,54.87,154838.0,16836.0,329724.0,-17.80,61.0,X-LC,3.17,2.0,0.11,2.32,14.11,X40,BTT,IT
76,TMPV,600.00,-1.33,-16.22,69.71,42.18,161431.0,-44835.0,231575.0,-24.30,20.0,X-LC,0.63,3.0,-0.28,1.63,0.00,XY24,NTT,AUTO
82,VBL,671.64,0.55,-9.39,49.74,35.67,142343.0,-29667.0,286175.0,-20.14,42.0,X-LC,5.95,4.0,-0.21,2.01,3.16,X40N,ATH,FMCG
43,ITC,452.00,-0.21,-1.61,12.24,10.44,24103.0,-3218.0,196920.0,-41.15,42.0,X-LC,0.83,5.0,-0.13,1.38,3.87,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,0.66,-12.66,108.02,81.67,88501.0,-11880.0,81930.0,7366.67,52.0,L-SC,18.77,271.0,-0.13,0.58,58.66,XR,NTT,CERAMICS
68,SONACOMS,806.63,-0.99,-13.36,61.07,39.54,53522.0,-13519.0,87640.0,-32.23,63.0,M-SC,6.70,220.0,-0.25,0.62,23.58,AR,ATH,AUTO
51,MASFIN,398.61,-0.32,-5.51,29.17,22.05,27006.0,-5400.0,92580.0,-18.70,49.0,H-SC,7.32,164.0,-0.20,0.65,34.57,XR,ATH,FINANCE
70,SURYODAY,216.00,-0.17,-19.96,53.73,23.03,77006.0,-35751.0,143320.0,55.70,48.0,H-SC,10.59,167.0,-0.46,1.01,42.20,XR,NTT,BANKS
39,INDIGOPNTS,1408.00,-0.08,-9.40,10.40,0.02,16447.0,-16416.0,158143.0,141.07,70.0,M-SC,6.66,234.0,-1.00,1.11,37.56,OX40N,NTT,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,-0.08,-9.40,10.40,0.02,16447.0,-16416.0,158143.0,141.07,70.0,M-SC,6.66,234.0,-1.00,1.11,37.56,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.26,0.24,0.87,19.90,20.95,50540.0,2200.0,253968.0,-0.21,77.0,X-LC,13.39,31.0,0.04,1.79,36.82,X40,ATH,PAINTS
40,INDUSINDBK,1800.00,0.53,-37.95,114.17,32.89,54694.0,-29300.0,47906.0,-691.63,60.0,L-MC,5.58,259.0,-0.54,0.34,31.93,XR,NTT,BANKS
12,BERGEPAINT,680.00,-0.74,2.44,18.26,21.14,42523.0,5536.0,232875.0,-4.75,61.0,X-MC,3.88,74.0,0.13,1.64,31.26,XY24,NTT,PAINTS
79,UNITDSPR,1644.00,-0.38,4.74,15.30,20.77,37521.0,11094.0,245238.0,-4.44,57.0,X-MC,3.55,62.0,0.30,1.72,11.98,X40N,NTT,BREWERIES


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.76
1,25,44.64
2,50,75.49


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.69
MC    29.25
LC    25.07
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.01
X40      21.21
X40N     11.84
XR        9.63
XY25      9.05
AR        8.98
OX40N     7.68
X200      1.84
SR        0.99
MH        0.78
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    23.96
X-MC    22.83
X-LC    19.96
M-SC    12.07
X-SC     8.24
H-MC     4.72
H-LC     2.95
L-SC     1.42
M-MC     1.36
M-LC     1.20
L-LC     0.96
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.79,-5.30,39.36
IT,13.73,-15.85,75.65
FINANCE,9.42,-15.62,64.47
MISC,7.43,-22.98,73.36
PAINTS,6.00,-8.29,24.38
ELECTRICAL,5.90,-11.69,52.60
INSURANCE,4.47,0.50,34.57
PHARMA,3.89,-1.19,33.58
AUTO,3.33,-21.57,73.69


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3055015.0,21
AR,1294133.0,10
XR,1293333.0,13
X40,1022684.0,14
X40N,762446.0,9
OX40N,700664.0,10
XY25,352348.0,6
SR,279464.0,2
MH,72644.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3569927.0,25
M-SC,1342665.0,15
X-MC,1237446.0,16
X-LC,922445.0,11
X-SC,746923.0,8
H-MC,407732.0,3
L-SC,260344.0,3
H-LC,136747.0,2
M-LC,121196.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1216603.0      6
           AR         878651.0      5
           XR         795931.0      7
M-SC       XY24       782984.0      6
X-MC       X40        448096.0      7
X-LC       X40        399586.0      5
X-MC       XY24       353813.0      3
H-SC       OX40N      326634.0      4
X-SC       X40N       296819.0      3
M-SC       OX40N      286379.0      5
H-SC       SR         279464.0      2
X-SC       XY24       275102.0      3
X-MC       X40N       253220.0      4
X-LC       XY24       234701.0      2
H-MC       AR         215920.0      2
X-LC       X40N       212407.0      2
H-MC       XY24       191812.0      1
X-MC       XY25       182317.0      2
X-SC       X40        175002.0      2
L-SC       XR         172693.0      2
M-SC       XR         148819.0      2
           AR         124483.0      2
M-LC       XR         121196.0      1
L-SC       OX40N       87651.0      1
X-LC       XY25        75751.0      2
H-LC       AR          75079.0      1
H-SC       MH          72644.0      1
H-LC       X200        61668.0      1
L-MC       XR          54694.0      1
M-MC       XY25        51997.0      1
L-LC       XY25        42283.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 32.0 seconds
